<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Bridge_Cracks/Bridge_Cracks_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================
# Enhanced Lightweight CNN for Bridge Cracks
# with num_images selection
# ==============================================

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------
# 2. Imports
# ----------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import cv2
import random
from sklearn.model_selection import train_test_split

# ----------------------------
# 3. Dataset Path & Parameters
# ----------------------------
dataset_dir = '/content/drive/MyDrive/Bridge_Cracks'  # contains Positive & Negative subfolders
num_images = 500  # <-- user can change this value
test_split = 0.2
batch_size = 32
seed = 22

# ----------------------------
# 4. Load All Images into Memory
# ----------------------------
data = []
labels = []

class_names = sorted(os.listdir(dataset_dir))
print("Classes:", class_names)

for idx, cls in enumerate(class_names):
    cls_folder = os.path.join(dataset_dir, cls)
    for fname in os.listdir(cls_folder):
        fpath = os.path.join(cls_folder, fname)
        img = cv2.imread(fpath)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        data.append(img)
        labels.append(idx)

# Shuffle & Limit number of images
combined = list(zip(data, labels))
random.seed(seed)
random.shuffle(combined)
data, labels = zip(*combined)
data = list(data)[:num_images]
labels = list(labels)[:num_images]

# Detect image size from first sample
img_height, img_width = data[0].shape[:2]
print(f"Detected image size: {img_height}x{img_width}")
data = np.array([cv2.resize(img, (img_width, img_height)) for img in data]) / 255.0
labels = np.array(labels)

# ----------------------------
# 5. Train-Test Split
# ----------------------------
X_train, X_val, y_train, y_val = train_test_split(
    data, labels, test_size=test_split, random_state=seed, stratify=labels)

print("Train size:", len(X_train), "Validation size:", len(X_val))

# ----------------------------
# 6. Build Enhanced Lightweight CNN
# ----------------------------
model = models.Sequential([
    layers.InputLayer(input_shape=(img_height, img_width, 3)),

    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ----------------------------
# 7. Train Model
# ----------------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=batch_size,
    verbose=1
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Classes: ['Negative', 'Positive']
Detected image size: 200x200
Train size: 400 Validation size: 100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 200, 200, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 200, 200, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 100, 100, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 100, 100, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 100, 100, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 50, 50, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 50, 50, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 50, 50, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,914 (433.26 KB)

 Trainable params: 110,466 (431.51 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 69s 5s/step - accuracy: 0.6463 - loss: 0.7885 - val_accuracy: 0.7300 - val_loss: 0.6562
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 80s 5s/step - accuracy: 0.7228 - loss: 0.5968 - val_accuracy: 0.7300 - val_loss: 0.6210
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 92s 6s/step - accuracy: 0.7772 - loss: 0.4790 - val_accuracy: 0.7300 - val_loss: 0.5865
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step - accuracy: 0.7476 - loss: 0.5730 - val_accuracy: 0.7300 - val_loss: 0.5759
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 0.7744 - loss: 0.4823 - val_accuracy: 0.7300 - val_loss: 0.5857
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 67s 5s/step - accuracy: 0.7793 - loss: 0.4844 - val_accuracy: 0.7300 - val_loss: 0.6355
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 80s 5s/step - accuracy: 0.7488 - loss: 0.4656 - val_accuracy: 0.7300 - val_loss: 0.6843
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 65s 5s/step - accuracy: 0.8333 - loss: 0.4052 - val_accuracy: 0.7300 - val_loss:

In [ ]:
# ----------------------------
# 9. Confusion Matrices (Train + Val)
# ----------------------------
def plot_confusion_matrix(y_true, y_prob, dataset_name):
    y_pred = (y_prob >= 0.5).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

    plt.figure(figsize=(5,4))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size":14})
    plt.title(f'{dataset_name} Confusion Matrix', fontsize=14)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.savefig(f'confusion_matrix_{dataset_name.lower()}.png')
    plt.show()

# Train CM
plot_confusion_matrix(y_train, y_prob_train, "Train")
# Validation CM
plot_confusion_matrix(y_val, y_prob_val, "Validation")

In [ ]:
# ----------------------------
# 10. Classification Report (Validation)
# ----------------------------
y_pred_val = (y_prob_val >= 0.5).astype(int)
report = classification_report(y_val, y_pred_val, target_names=class_names, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report.to_csv('classification_report_enhanced.csv', index=True)
df_report

In [ ]:
# ----------------------------
# 11. Accuracy and Loss Plots
# ----------------------------
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('accuracy_loss_enhanced.png')
plt.show()

# ----------------------------
# 12. Save Model & Weights
# ----------------------------
model.save('enhanced_lightcnn_bridgecracks_model.h5')
model.save_weights('enhanced_lightcnn_bridgecracks_weights.h5')

In [ ]:
# ----------------------------
# 8. ROC Curves
# ----------------------------
y_prob_train = model.predict(X_train)[:,1]
y_prob_val = model.predict(X_val)[:,1]

fpr_train, tpr_train, _ = roc_curve(y_train, y_prob_train)
roc_auc_train = auc(fpr_train, tpr_train)

fpr_val, tpr_val, _ = roc_curve(y_val, y_prob_val)
roc_auc_val = auc(fpr_val, tpr_val)

plt.figure(figsize=(5,4))
plt.plot(fpr_train, tpr_train, color='blue', label=f'Train AUC = {roc_auc_train:.2f}')
plt.plot(fpr_val, tpr_val, color='red', label=f'Val AUC = {roc_auc_val:.2f}')
plt.plot([0,1], [0,1], color='gray', linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curves')
plt.legend()
plt.grid()
plt.savefig('roc_curves_enhanced.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import random

# Paths for Positive and Negative subfolders
positive_dir = os.path.join(dataset_dir, "Positive")
negative_dir = os.path.join(dataset_dir, "Negative")

# Pick 4 random samples from each class
positive_samples = random.sample(os.listdir(positive_dir), 4)
negative_samples = random.sample(os.listdir(negative_dir), 4)

# Plot figure
fig, axes = plt.subplots(2, 4, figsize=(12,6))

# First row: Positive cracks
for i, img_name in enumerate(positive_samples):
    img_path = os.path.join(positive_dir, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[0, i].imshow(img)
    axes[0, i].set_title("Positive")
    axes[0, i].axis("off")

# Second row: Negative (no cracks)
for i, img_name in enumerate(negative_samples):
    img_path = os.path.join(negative_dir, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[1, i].imshow(img)
    axes[1, i].set_title("Negative")
    axes[1, i].axis("off")

plt.suptitle("Sample Bridge Crack Dataset Images", fontsize=16)
plt.tight_layout()
plt.savefig("sample_bridge_crack_images.png")
plt.show()
